# 도매꾹 베스트 상품 크롤링 프로그램

## 프로그램 설명
도매꾹 웹사이트에서 카테고리별 베스트 상품 정보를 자동으로 수집하고 구글 드라이브에 업로드하는 프로그램입니다.

## 주요 기능
- 도매꾹 카테고리별 베스트 상품 정보 수집
- 구글 드라이브 자동 업로드
- 여러 카테고리 지원 (전체, 패션잡화, 의류, 출산/유아동, 가구/생활, 스포츠/건강, 가전/휴대폰)

## 사용 방법
1. 프로그램 실행
2. 자동으로 카테고리별 베스트 상품 수집
3. 구글 드라이브에 자동 업로드

## 필요 라이브러리
- beautifulsoup4: HTML 파싱
- pandas: 데이터 처리
- requests: HTTP 요청
- google-api-python-client: 구글 드라이브 API

## 주의사항
- Google Drive API 인증이 필요합니다 (client_secret.json 파일 필요)
- 도매꾹 웹사이트 구조 변경 시 수정이 필요할 수 있습니다

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import requests
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os

# Google Drive 폴더 ID
GOOGLE_DRIVE_FOLDER_ID = "15EbkbKbhoLhXYTPMAv8FusO67H4EZlL4"

def upload_to_google_drive(file_path, folder_id):
    SCOPES = ['https://www.googleapis.com/auth/drive.file']
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)
    file_metadata = {'name': os.path.basename(file_path), 'parents': [folder_id]}
    media = MediaFileUpload(file_path, resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f"파일이 Google Drive에 업로드되었습니다. File ID: {file.get('id')}")
    return file.get('id')

# 카테고리 URL 목록
categories = {
    "전체": "https://domeggook.com/main/item/itemPopular.php?cat=01_00",
    "패션잡화_화장품": "https://domeggook.com/main/item/itemPopular.php?cat=01_01",
    "의류_언더웨어": "https://domeggook.com/main/item/itemPopular.php?cat=01_02",
    "출산_유아동_완구": "https://domeggook.com/main/item/itemPopular.php?cat=01_03",
    "가구_생활_취미": "https://domeggook.com/main/item/itemPopular.php?cat=01_04",
    "스포츠_건강_식품": "https://domeggook.com/main/item/itemPopular.php?cat=01_05",
    "가전_휴대폰_산업": "https://domeggook.com/main/item/itemPopular.php?cat=01_06",
}

def scrape_category(url):
    """카테고리별 데이터를 수집하는 함수"""
    response = requests.get(url)
    response.encoding = 'euc-kr'
    soup = BeautifulSoup(response.text, "html.parser")
    
    data = []
    
    for product in soup.find_all("li", class_=["topBigLi", "topSmallLi", "topMiddleLi"]):
        # 순위
        rank = int(product.find("span", class_="grade").get_text(strip=True))
        
        # 순위 변화 및 기호 처리
        grade_up_down = product.find("span", class_="gradeUpDown").get_text(strip=True)
        if "NEW" in grade_up_down:
            rank_change = "NEW"
            rank_symbol = "▲"
        elif len(grade_up_down) > 1:
            rank_symbol = grade_up_down[0]
            try:
                rank_change = int(grade_up_down[1:]) if rank_symbol == "▲" else -int(grade_up_down[1:])
            except ValueError:
                rank_change = 0
        else:
            rank_symbol = "-"
            rank_change = 0

        # 상품명
        product_name_tag = product.find("div", class_="title")
        product_name = product_name_tag.find("a").get_text(strip=True) if product_name_tag else "N/A"
        
        # 가격
        price_tag = product.find("div", class_="price")
        price_text = price_tag.get_text(strip=True).replace("원", "").replace(",", "") if price_tag else "0"
        price = int(price_text) if price_text.isdigit() else 0
        
        # 최소 주문 수량
        qty_tag = product.find("div", class_="qty")
        qty_text = qty_tag.find("b").get_text(strip=True).replace(",", "") if qty_tag else "0"
        min_qty = int(qty_text) if qty_text.isdigit() else 0
        
        # 상품 URL
        product_link = product_name_tag.find("a")["href"] if product_name_tag else ""
        product_url = f"https://domeggook.com{product_link}"

        # 이미지 URL
        image_tag = product.find("div", class_="info").find("img")
        image_url = image_tag["src"] if image_tag else ""
        
        data.append({
            "순위": rank,
            "순위 변화": rank_change,
            "순위 변동기호": rank_symbol,
            "상품명": product_name,
            "가격": price,
            "최소 주문수량": min_qty,
            "상품 URL": product_url,
            "이미지 URL": image_url
        })
    
    return pd.DataFrame(data)

def main():
    # 오늘 날짜를 포함한 파일명 설정
    today_date = datetime.datetime.today().strftime("%Y%m%d")
    
    # 각 카테고리 데이터 수집 및 개별 CSV 파일 저장
    for category_name, url in categories.items():
        print(f"{category_name} 카테고리 데이터를 수집 중...")
        df = scrape_category(url)
        
        # 카테고리별 CSV 파일명 설정
        file_name = f"{category_name}_도매꾹_베스트상품_{today_date}.csv"
        df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"{file_name} 파일로 저장되었습니다.")
        
        # Google Drive에 업로드
        file_id = upload_to_google_drive(file_name, GOOGLE_DRIVE_FOLDER_ID)
    
    print("모든 카테고리의 CSV 파일이 Google Drive에 업로드되었습니다.")

if __name__ == "__main__":
    main()


전체 카테고리 데이터를 수집 중...
전체_도매꾹_베스트상품_20241107.csv 파일로 저장되었습니다.
파일이 Google Drive에 업로드되었습니다. File ID: 1OQ_htNTZEeZHeGknYJ7se6-4V8HzHYmh
패션잡화_화장품 카테고리 데이터를 수집 중...
패션잡화_화장품_도매꾹_베스트상품_20241107.csv 파일로 저장되었습니다.
파일이 Google Drive에 업로드되었습니다. File ID: 1BdCtXpd7bWhOPDQyPHpwC_r3dm8CZHmh
의류_언더웨어 카테고리 데이터를 수집 중...
의류_언더웨어_도매꾹_베스트상품_20241107.csv 파일로 저장되었습니다.
파일이 Google Drive에 업로드되었습니다. File ID: 1YMDufU8FncR4hvDKce38kkk6-sag-43v
출산_유아동_완구 카테고리 데이터를 수집 중...
출산_유아동_완구_도매꾹_베스트상품_20241107.csv 파일로 저장되었습니다.
파일이 Google Drive에 업로드되었습니다. File ID: 1ogIegMHYFMU-iJtuHbVo-oWDZO6VGegB
가구_생활_취미 카테고리 데이터를 수집 중...
가구_생활_취미_도매꾹_베스트상품_20241107.csv 파일로 저장되었습니다.
파일이 Google Drive에 업로드되었습니다. File ID: 1gt7qgQYQSETVqJbFWsmNx3pk7gmDUgoA
스포츠_건강_식품 카테고리 데이터를 수집 중...
스포츠_건강_식품_도매꾹_베스트상품_20241107.csv 파일로 저장되었습니다.
파일이 Google Drive에 업로드되었습니다. File ID: 1_cpwD6rSYz5LCtZevuF-OaHoaZN5SEIZ
가전_휴대폰_산업 카테고리 데이터를 수집 중...
가전_휴대폰_산업_도매꾹_베스트상품_20241107.csv 파일로 저장되었습니다.
파일이 Google Drive에 업로드되었습니다. File ID: 1UcMk-wffrYiQ-ws4SZNPa1ZPqdM6Sj-5
모든 카테고리